### Input path of file

In [1]:
import cv2
import pymongo
import face_recognition

# Connect to MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017")
db = client["face_recognition"]
collection = db["face recognition"]

# Function to extract face encodings using face_recognition
def extract_face_encodings(image):
    face_locations = face_recognition.face_locations(image)
    face_encodings = face_recognition.face_encodings(image, face_locations)
    return face_encodings

# Main program
def main():
    # Take the image file path as input
    image_path = input("Enter the path of the image: ")

    # Read the image
    image = cv2.imread(image_path)

    # Check if the image was read successfully
    if image is None:
        print("Failed to read the image")
        return

    # Take a name as input
    name = input("Enter a name: ")

    # Extract face encodings using face_recognition
    face_encodings = extract_face_encodings(image)

    # Check if any face encodings were extracted
    if len(face_encodings) == 0:
        print("No face detected in the image")
        return

    # Generate a unique ID (7-digit number)
    id = collection.count_documents({}) + 1

    # Create a document with the face encodings, name, and ID
    document = {"id": id, "encodings": face_encodings[0].tolist(), "name": name}

    # Insert the document into the collection
    result = collection.insert_one(document)

    # Print the inserted document's ID
    print("Document inserted with ID:", result.inserted_id)


if __name__ == '__main__':
    main()


Document inserted with ID: 64818e9e1d048e4461d3f9c6


### Choose image from files

In [14]:
import cv2
import pymongo
import face_recognition
from PyQt5.QtWidgets import QApplication, QFileDialog

# Connect to MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017")
db = client["face_recognition"]
collection = db["face recognition"]

def extract_face_encodings(image):
    face_encodings = face_recognition.face_encodings(image)

    if len(face_encodings) > 0:
        return face_encodings[0]
    else:
        return None


def main():
    # Create a Qt application instance
    app = QApplication([])

    # Open a file dialog to select an image
    dialog = QFileDialog()
    dialog.setFileMode(QFileDialog.ExistingFile)
    dialog.setNameFilter("Images (*.png *.jpg *.jpeg)")

    if dialog.exec_():
        # Get the selected file path
        image_path = dialog.selectedFiles()[0]

        # Read the image
        image = cv2.imread(image_path)

        # Check if the image was read successfully
        if image is None:
            print("Failed to read the image")
            return

        # Take a name as input
        name = input("Enter a name: ")

        # Extract face encodings using face_recognition
        face_encodings = extract_face_encodings(image)

        if face_encodings is None:
            print("No face detected in the image")
            return

        # Generate a unique ID (7-digit number)
        id = collection.count_documents({}) + 1

        # Create a document with the face encodings, name, and ID
        document = {"id": id, "encodings": face_encodings.tolist(), "name": name}

        # Insert the document into the collection
        result = collection.insert_one(document)

        # Print the inserted document's ID
        print("Document inserted with ID:", result.inserted_id)

    # Quit the application
    app.quit()


if __name__ == '__main__':
    main()


Document inserted with ID: 6481918b63cefab1b48ec2e7


### Capture image using webcam

In [ ]:
import cv2
import pymongo
import matplotlib.pyplot as plt
import face_recognition

# Connect to MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017")
db = client["face_recognition"]
collection = db["face recognition"]

# Function to capture an image
def capture_image():
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Failed to open camera")
        return None
    ret, frame = cap.read()
    cap.release()
    return frame

# Function to extract face encodings using face_recognition
def extract_face_encodings(image):
    face_locations = face_recognition.face_locations(image)
    face_encodings = face_recognition.face_encodings(image, face_locations)
    return face_encodings

# Function to save data to MongoDB
def save_to_mongodb(face_encodings, name):
    id = collection.count_documents({}) + 1
    document = {"id": id, "encodings": face_encodings[0].tolist(), "name": name}
    result = collection.insert_one(document)
    print("Document inserted with ID:", result.inserted_id)

def main():
    image = capture_image()
    if image is None:
        print("Failed to capture image")
        return

    # Display the captured image
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

    # Ask for confirmation to upload the image
    confirmation = input("Do you want to upload this image to the database? (yes/no): ")
    if confirmation.lower() == "yes":
        name = input("Enter a name: ")
        face_encodings = extract_face_encodings(image)
        if len(face_encodings) == 0:
            print("No face detected in the image")
            return
        save_to_mongodb(face_encodings[0], name)
        print("Face encodings saved to MongoDB with name:", name)
    else:
        print("Image not uploaded.")

if __name__ == '__main__':
    main()
